[구글 코랩(Colab)에서 실행하기](https://colab.research.google.com/github/lovedlim/tensorflow/blob/main/Part%204/4.7_distillation.ipynb)

## Import

In [ ]:
import tensorflow as tf
import numpy as np
from google.colab.patches import cv2_imshow
from tqdm import tqdm

## 주요 파라미터 설정

In [ ]:
#@title 파라미터 설정
t_ephoc = 10 #@param {type:"slider", min:1, max:100, step:1}
s_ephoc = 5 #@param {type:"slider", min:1, max:100, step:1}
learning_rate = 0.01 
batch_size = 64 #@param [32, 64, 128, 256] {type:"raw"}
temperature = 3 #@param {type:"slider", min:1, max:10, step:1}
alpha = 0.5 #@param {type:"slider", min:0.1, max:0.9, step:0.1}

## 데이터셋

In [ ]:
# MNIST 데이터셋 가져오기

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = x_train.astype("float32") / 255.0
x_train = np.reshape(x_train, (-1, 28, 28, 1))

x_test = x_test.astype("float32") / 255.0
x_test = np.reshape(x_test, (-1, 28, 28, 1))

## 모델 정의

In [ ]:
# teacher 모델
i=tf.keras.Input(shape=(28, 28, 1))
out=tf.keras.layers.Conv2D(256, (3, 3), strides=(2, 2), padding="same")(i)
out=tf.keras.layers.LeakyReLU(alpha=0.2)(out)
out=tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding="same")(out)
out=tf.keras.layers.Conv2D(512, (3, 3), strides=(2, 2), padding="same")(out)
out=tf.keras.layers.Flatten()(out)
out=tf.keras.layers.Dense(10)(out)
t_model=tf.keras.Model(inputs=[i],outputs=[out])

t_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 14, 14, 256)       2560      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 14, 14, 256)       0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 256)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 512)         1180160   
_________________________________________________________________
flatten (Flatten)            (None, 25088)             0         
_________________________________________________________________
dense (Dense)                (None, 10)                250890

In [ ]:
# student 모델
i=tf.keras.Input(shape=(28, 28, 1))
out=tf.keras.layers.Flatten()(i)
out=tf.keras.layers.Dense(28)(out)
out=tf.keras.layers.Dense(10)(out)

s_model_1=tf.keras.Model(inputs=[i],outputs=[out])
s_model_2=tf.keras.models.clone_model(s_model_1)

s_model_1.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 28)                21980     
_________________________________________________________________
dense_2 (Dense)              (None, 10)                290       
Total params: 22,270
Trainable params: 22,270
Non-trainable params: 0
_________________________________________________________________


## 모델 컴파일

In [ ]:
# teacher 모델
t_model.compile(tf.keras.optimizers.Adam(learning_rate),
                tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

# student 모델 (distillation 적용)
s_model_1.compile(tf.keras.optimizers.Adam(learning_rate),
                tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

# 비교 모델 (distillation 미적용)
s_model_2.compile(tf.keras.optimizers.Adam(learning_rate),
                tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

## 모델 훈련

In [ ]:
# teacher 모델
t_model.fit(x_train, y_train,batch_size=batch_size,epochs=t_ephoc)

Epoch 1/10
938/938 [==============================] - 9s 7ms/step - loss: 1.8960 - sparse_categorical_accuracy: 0.8368
Epoch 2/10
938/938 [==============================] - 7s 7ms/step - loss: 17.1700 - sparse_categorical_accuracy: 0.9142
Epoch 3/10
938/938 [==============================] - 7s 7ms/step - loss: 4.8890 - sparse_categorical_accuracy: 0.9621
Epoch 4/10
938/938 [==============================] - 7s 7ms/step - loss: 5.8202 - sparse_categorical_accuracy: 0.9633
Epoch 5/10
938/938 [==============================] - 7s 7ms/step - loss: 10.4369 - sparse_categorical_accuracy: 0.9643
Epoch 6/10
938/938 [==============================] - 7s 7ms/step - loss: 7.3991 - sparse_categorical_accuracy: 0.9717
Epoch 7/10
938/938 [==============================] - 7s 7ms/step - loss: 9.6172 - sparse_categorical_accuracy: 0.9706
Epoch 8/10
938/938 [==============================] - 7s 7ms/step - loss: 8.7060 - sparse_categorical_accuracy: 0.9752
Epoch 9/10
938/938 [==========================

In [ ]:
# student 손실함수
s_loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
# distillation 손실함수
d_loss = tf.keras.losses.KLDivergence()

In [ ]:
x_train.shape

(60000, 28, 28, 1)

In [ ]:
batch_count =x_train.shape[0]//batch_size

opt = tf.keras.optimizers.Adam(learning_rate)

for e in range(s_ephoc):
    for _ in range(batch_count):
        batch_num=np.random.randint(0, x_train.shape[0], size=batch_size)
        t_pred = t_model.predict(x_train[batch_num])

        with tf.GradientTape() as tape:
            s_pred_1 = s_model_1(x_train[batch_num])
            student_loss = s_loss(y_train[batch_num], s_pred_1)
            distillation_loss = d_loss(
                tf.nn.softmax(t_pred / temperature, axis=1),
                tf.nn.softmax(s_pred_1 / temperature, axis=1),
            )
            loss = alpha * student_loss + (1 - alpha) * distillation_loss

        vars = s_model_1.trainable_variables
        grad = tape.gradient(loss, vars)
        opt.apply_gradients(zip(grad, vars))

        with tf.GradientTape() as tape:
            s_pred_2 = s_model_2(x_train[batch_num])
            student_loss = s_loss(y_train[batch_num], s_pred_2)
        vars = s_model_2.trainable_variables
        grad = tape.gradient(student_loss, vars)
        opt.apply_gradients(zip(grad, vars))

    print("에포크 {}".format(e))
    print("선생님께 배운 경우")
    s_model_1.evaluate(x_test, y_test)
    print("혼자 공부한 경우")
    s_model_2.evaluate(x_test, y_test)
    print("\n")

에포크 0
선생님께 배운 경우
313/313 [==============================] - 1s 2ms/step - loss: 0.5909 - sparse_categorical_accuracy: 0.8753
혼자 공부한 경우
313/313 [==============================] - 1s 2ms/step - loss: 0.4053 - sparse_categorical_accuracy: 0.8880


에포크 1
선생님께 배운 경우
313/313 [==============================] - 0s 1ms/step - loss: 0.4574 - sparse_categorical_accuracy: 0.9058
혼자 공부한 경우
313/313 [==============================] - 0s 2ms/step - loss: 0.3765 - sparse_categorical_accuracy: 0.8931


에포크 2
선생님께 배운 경우
313/313 [==============================] - 1s 2ms/step - loss: 0.4398 - sparse_categorical_accuracy: 0.9133
혼자 공부한 경우
313/313 [==============================] - 1s 2ms/step - loss: 0.3225 - sparse_categorical_accuracy: 0.9103


에포크 3
선생님께 배운 경우
313/313 [==============================] - 1s 2ms/step - loss: 0.4557 - sparse_categorical_accuracy: 0.9089
혼자 공부한 경우
313/313 [==============================] - 0s 2ms/step - loss: 0.3301 - sparse_categorical_accuracy: 0.9087


에포크 4
선생님께 배운 경우
313